In [1]:
import requests
from POP import PromptFunction, load_prompt, get_text_snapshot
from openai import OpenAI
from dotenv import load_dotenv
from os import getenv
import json

## Perplexity

In [3]:
url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": "Be precise and concise."
        },
        {
            "role": "user",
            "content": "Why is winter cold?"
        }
    ],
    #"max_tokens": "Optional",
    "temperature": 0.2,
    "top_p": 0.9,
    #"search_domain_filter": ["perplexity.ai"],
    "return_images": False,
    "return_related_questions": False,
    "search_recency_filter": "month",
    "top_k": 0,
    "stream": False,
    "presence_penalty": 0,
    "frequency_penalty": 1,
    "response_format": None
}
headers = {
    "Authorization": f"Bearer {getenv('PERPLEXITY_API_KEY')}",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"id": "97ddd261-dbf7-4704-8cf4-1feee08c2ea2", "model": "sonar", "created": 1738720949, "usage": {"prompt_tokens": 10, "completion_tokens": 142, "total_tokens": 152}, "citations": ["https://www.youtube.com/watch?v=szhD4tn0-0Y", "https://site.extension.uga.edu/climate/2025/01/why-has-this-la-nina-winter-been-so-cold/"], "object": "chat.completion", "choices": [{"index": 0, "finish_reason": "stop", "message": {"role": "assistant", "content": "Winter can be cold due to several factors:\n\n1. **Seasonal Changes**: During winter, the Earth's axis is tilted away from the Sun, reducing the amount of sunlight and warmth received by the Northern Hemisphere, leading to colder temperatures.\n\n2. **Weather Patterns**: Phenomena like La Ni\u00f1a can influence weather patterns, sometimes causing cold air to move southward from the Arctic. Weak circulation around the North Pole can allow lobes of cold air to penetrate into mid-latitudes[2].\n\n3. **Polar Vortex Disruptions**: Warming Arctic tempera

In [2]:
client_perplexity = OpenAI(base_url="https://api.perplexity.ai", api_key=getenv("PERPLEXITY_API_KEY"))

messages = [
                {
                    "role": "system",
                    "content": (
                        "You are an artificial intelligence assistant and you need to "
                        "engage in a helpful, detailed, polite conversation with a user."
                    ),
                },
                {
                    "role": "user",
                    "content": "What's the weather for east america? Include MA, NY, and PA.",
                },
            ]
response = client_perplexity.chat.completions.create(model="sonar-pro", messages=messages)
answer = response.choices[0].message.content
citations = response.citations  # usually a list of strings
print("Answer:\n", answer)
print("Citations:\n", citations)

Answer:
 Based on the search results, here's an overview of the weather for eastern America, focusing on Massachusetts (MA), New York (NY), and Pennsylvania (PA) in February 2025:

Massachusetts:
The weather in Massachusetts is expected to be cold and snowy in February 2025. Average temperatures range from -5.7°C (21.7°F) at night to 3.2°C (37.7°F) during the day[1][7]. Boston specifically is forecasted to have daytime highs of 3°C (37°F) and nighttime lows of -7°C (19°F)[10]. Snowfall is likely, with an average of 15 snowy days expected throughout the month[1]. Precipitation is anticipated to be around 87-93mm (3.4-3.7 inches)[7][10].

New York:
While specific data for New York wasn't provided in the search results, we can infer that the weather will be similar to neighboring states. Expect cold temperatures, potential snowfall, and occasional rain or wintry mix events.

Pennsylvania:
Pennsylvania is likely to experience cold temperatures in February 2025, with a mix of snow and rain 

In [ ]:
# generate a loop from 1 to 10 print their squares
for i in range(1, 11):
    print(f"{i} squared is {i**2}")

## Deepseek (Local)

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
#model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Generate a response
prompt = "What is the capital of the United States?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
response = tokenizer.decode(outputs[0])
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>What is the capital of the United States? What is the largest planet in the solar system? What is the largest country in the world? What is the largest planet in the solar system? What is the largest country in the world? What is the capital of the United States? What is the largest planet in the solar system? What is the capital of the United States? What is the largest planet in the solar system? What is the largest planet in the solar system? What is the capital


## Story Outliner

In [3]:
pf_outliner = PromptFunction(load_prompt("story_outliner_prompt.md"))
print(pf_outliner.prompt)

## Bedtime Story Outline Generator (JSON Structured Output)

**Objective:**  
Generate a comprehensive and imaginative outline for a bedtime story that can later be expanded into full chapters. The output must be in valid JSON format following the structure provided below.

---

### User's Request:
<<<user_request>>>

---

### Instructions:

Please generate the bedtime story outline using the JSON structure detailed below:

1. **Story Overview** (`storyOverview` object):
   - **title**: Provide a creative and engaging title for the story.
   - **summary**: Write a brief summary capturing the main plot, setting, and overall mood of the story.

2. **Main Character Details** (`mainCharacter` object):
   - **name**: Name of the main character.
   - **background**: A short background story.
   - **personality**: Description of personality traits, motivations, strengths, and vulnerabilities.
   - **appearanceAndCostumes**: Details of physical appearance and distinctive clothing or costumes (

In [17]:
fmt = {
  "name": "Bedtime_Story_Outline",
  "schema": {
    "type": "object",
    "properties": {
        "storyOverview": {
        "type": "object",
        "properties": {
            "title": { "type": "string", "description": "A creative and engaging title for the story." },
            "summary": { "type": "string", "description": "A brief summary capturing the main plot, setting, and overall mood." }
        },
        "required": ["title", "summary"]
        },
        "mainCharacter": {
        "type": "object",
        "properties": {
            "name": { "type": "string", "description": "Name of the main character." },
            "background": { "type": "string", "description": "A short background story for the main character." },
            "personality": { "type": "string", "description": "Description of personality traits, motivations, strengths, and vulnerabilities." },
            "appearanceAndCostumes": { "type": "string", "description": "Details of the character's physical appearance and distinctive clothing or costumes." }
        },
        "required": ["name", "background", "personality", "appearanceAndCostumes"]
        },
        "setting": {
        "type": "object",
        "properties": {
            "primarySetting": { "type": "string", "description": "Description of the main setting where the story takes place." },
            "atmosphereDetails": { "type": "string", "description": "Sensory details (sights, sounds, smells) that bring the setting to life." }
        },
        "required": ["primarySetting", "atmosphereDetails"]
        },
        "chapters": {
        "type": "array",
        "description": "Array of chapter outlines.",
        "items": {
            "type": "object",
            "properties": {
            "chapterTitle": { "type": "string", "description": "A catchy title for the chapter." },
            "chapterSummary": { "type": "string", "description": "A brief overview of what happens in the chapter." },
            "keyEvents": {
                "type": "array",
                "description": "List of main events, actions, or conflicts in the chapter.",
                "items": { "type": "string" }
            },
            "characterDevelopment": { "type": "string", "description": "Insights, challenges, or growth for the main character during the chapter." },
            "supportingElements": { "type": "string", "description": "Additional characters, magical elements, or important items introduced." },
            "chapterConclusion": { "type": "string", "description": "Explanation of how the chapter ends and sets up the next chapter." },
            "currentChapter": { "type": "integer", "description": "The current chapter number."}
            },
            "required": ["chapterTitle", "chapterSummary", "keyEvents", "characterDevelopment", "supportingElements", "chapterConclusion"]
        }
        },
        "total_chapter": {
            "type": "integer",
            "description": "The total number of chapters in the story."
        },
        "supportingCharacters": {
        "type": "array",
        "description": "Array of supporting characters or antagonists.",
        "items": {
            "type": "object",
            "properties": {
            "name": { "type": "string", "description": "Name of the character." },
            "description": { "type": "string", "description": "A brief description of the character." },
            "costumesAndTraits": { "type": "string", "description": "Details about the character's appearance, costumes, and defining characteristics." }
            },
            "required": ["name", "description", "costumesAndTraits"]
        }
        },
        "themes": {
        "type": "object",
        "properties": {
            "coreThemes": {
            "type": "array",
            "description": "An array of the central themes (e.g., friendship, bravery, kindness).",
            "items": { "type": "string" }
            },
            "lessons": { "type": "string", "description": "The moral or educational lessons that the story will convey." }
        },
        "required": ["coreThemes", "lessons"]
        },
        "additionalElements": {
        "type": "object",
        "properties": {
            "recurringMotifs": { "type": "string", "description": "Symbols or motifs that recur throughout the story." },
            "specialFeatures": { "type": "string", "description": "Whimsical or fantastical elements that add to the bedtime story magic." }
        },
        "required": ["recurringMotifs", "specialFeatures"]
        }
    },
    "required": ["storyOverview", "mainCharacter", "setting", "chapters", "total_chapter", "supportingCharacters", "themes", "additionalElements"]
  }
  
}


In [31]:
user_request = "小红帽"
outline = pf_outliner.execute(user_request=user_request, model = "gpt-4o", fmt = fmt)
print(outline)

{"storyOverview":{"title":"Little Red's Enchanted Journey","summary":"A delightful tale of Little Red, a spirited young girl, who embarks on an adventurous journey through the Enchanted Woods to visit her grandmother, encountering magical creatures and learning valuable life lessons along the way."},"mainCharacter":{"name":"Little Red","background":"A lively and adventurous girl living in a quaint village at the edge of the Enchanted Woods.","personality":"Curious, brave, and kind-hearted, Little Red is always eager to explore and help others, but her impulsive nature sometimes leads her into tricky situations.","appearanceAndCostumes":"Wears a signature red hooded cloak, gifted by her grandmother, with a woven basket filled with treats. Her attire is simple yet charming, with a playful touch of magic as the cloak seems to shimmer in the forest light."},"setting":{"primarySetting":"The Enchanted Woods, a mystical forest filled with towering trees, hidden creatures, and magical pathways

In [32]:
outline = json.loads(outline)
outline

{'storyOverview': {'title': "Little Red's Enchanted Journey",
  'summary': 'A delightful tale of Little Red, a spirited young girl, who embarks on an adventurous journey through the Enchanted Woods to visit her grandmother, encountering magical creatures and learning valuable life lessons along the way.'},
 'mainCharacter': {'name': 'Little Red',
  'background': 'A lively and adventurous girl living in a quaint village at the edge of the Enchanted Woods.',
  'personality': 'Curious, brave, and kind-hearted, Little Red is always eager to explore and help others, but her impulsive nature sometimes leads her into tricky situations.',
  'appearanceAndCostumes': 'Wears a signature red hooded cloak, gifted by her grandmother, with a woven basket filled with treats. Her attire is simple yet charming, with a playful touch of magic as the cloak seems to shimmer in the forest light.'},
 'setting': {'primarySetting': 'The Enchanted Woods, a mystical forest filled with towering trees, hidden creat

In [33]:
pf_chapter_writer = PromptFunction('''
Generate the chapter of the bedtime story with the following details:
- **User Request:** <<<user_request>>>
- **Story Setting:** <<<story_setting>>>
- **Story Title:** <<<story_title>>>
- **Chapter Number:** <<<chapter_number>>>

For this chapter, please include:

1. **Chapter Title:**  
   Provide a catchy and engaging title for this chapter.

2. **Chapter Summary:**  
   Write a brief summary describing what happens in this chapter.

3. **Paragraph:**  
   List the main events, actions, or conflicts that occur in the chapter (use bullet points).
                                   
Ensure the tone is soothing, imaginative, and appropriate for a bedtime story.

''')

In [34]:
outline['storyOverview']['title']

"Little Red's Enchanted Journey"

In [35]:
outline['total_chapter']

4

In [36]:
paragraph = pf_chapter_writer.execute(user_request = user_request,
                          story_setting = outline,
                          story_title = outline['storyOverview']['title'],
                          chapter_number = 1,
                          model = "gpt-4o")

print(paragraph)

**Chapter 1: Into the Enchanted Woods**

**Chapter Summary:**  
In this opening chapter, Little Red begins her exciting journey through the Enchanted Woods to visit her beloved grandmother. As she steps into the magical forest, she encounters a friendly squirrel named Nutty, who offers her some wise advice. Despite the mystical surprises that lie ahead, Little Red's adventurous spirit is ignited, and she eagerly follows the shimmering path deeper into the woods.

**Paragraph:**

- Little Red receives a warm farewell from her mother, who hands her a woven basket filled with delicious treats for her grandmother.
- With her signature red hooded cloak shimmering in the forest light, Little Red sets off on her journey, feeling the thrill of adventure coursing through her.
- As she enters the Enchanted Woods, she is greeted by the lively sounds of chirping birds and the fragrant scent of blooming wildflowers.
- Little Red encounters Nutty the Squirrel, a chatty and friendly creature, who war

In [26]:
for i in range(1, outline['total_chapter']+1):
    paragraph = pf_chapter_writer.execute(user_request = user_request,
                          story_setting = outline,
                          story_title = outline['storyOverview']['title'],
                          chapter_number = i,
                          model = "gpt-4o")
    print(paragraph)

**Chapter Title:** A Whisper of Trouble

**Chapter Summary:**  
In the heart of Dreamwood Forest, a brave young rabbit named Thumper learns of a dragon's presence near the village, stirring fear among the woodland creatures. As panic spreads, Thumper's sense of justice and courage compel him to investigate the rumors himself. Guided by the wisdom of a wise old tortoise and a magical compass, Thumper sets off on a journey to uncover the truth behind the dragon's appearance.

**Paragraph:**

- Thumper, with his bright, curious eyes, overhears a hushed conversation between two sparrows about a dragon sighting.
- The news spreads like wildfire through the forest, leaving the animals in a state of panic and disarray.
- Driven by a desire to protect his home and loved ones, Thumper resolves to uncover the truth about the dragon.
- He seeks guidance from the wise old Tortoise, who lives by the shimmering stream deep within the forest.
- The Tortoise, with ancient wisdom, provides Thumper with

## GPT Auto-JSON formatter

GPT Auto formatter is a prompt function that outputs the `fmt` accepted by `PromptFunction` objects, based on given Natural Language description. It can also be used in auto task chains.

In [21]:
prompt_path = "prompts/json_formatter_prompt.md"
#prompt_path = "prompts/openai-json_schema_generator.md"
pf_formatter = PromptFunction(sys_prompt=load_prompt(prompt_path),
                              prompt="I need help generating a JSON schema for a specific format. Can you assist me with this task? Here's the description: \n<<<user_description>>>",)

In [22]:
## make sure the return is a json object, we create a flexible json schema to allow any key value pairs
formatter_fmt = {
  "name": "Any_name",
  "schema": {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": True,
    "properties": {}
    }
}

In [13]:
formatter_fmt = {
  "name": "metaschema",
  "schema": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name of the schema"
      },
      "type": {
        "type": "string",
        "enum": [
          "object",
          "array",
          "string",
          "number",
          "boolean",
          "null"
        ]
      },
      "properties": {
        "type": "object",
        "additionalProperties": {
          "$ref": "#/$defs/schema_definition"
        }
      },
      "items": {
        "anyOf": [
          {
            "$ref": "#/$defs/schema_definition"
          },
          {
            "type": "array",
            "items": {
              "$ref": "#/$defs/schema_definition"
            }
          }
        ]
      },
      "required": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "additionalProperties": {
        "type": "boolean"
      }
    },
    "required": [
      "type"
    ],
    "additionalProperties": False,
    "if": {
      "properties": {
        "type": {
          "const": "object"
        }
      }
    },
    "then": {
      "required": [
        "properties"
      ]
    },
    "$defs": {
      "schema_definition": {
        "type": "object",
        "properties": {
          "type": {
            "type": "string",
            "enum": [
              "object",
              "array",
              "string",
              "number",
              "boolean",
              "null"
            ]
          },
          "properties": {
            "type": "object",
            "additionalProperties": {
              "$ref": "#/$defs/schema_definition"
            }
          },
          "items": {
            "anyOf": [
              {
                "$ref": "#/$defs/schema_definition"
              },
              {
                "type": "array",
                "items": {
                  "$ref": "#/$defs/schema_definition"
                }
              }
            ]
          },
          "required": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "additionalProperties": {
            "type": "boolean"
          }
        },
        "required": [
          "type"
        ],
        "additionalProperties": False,
        "if": {
          "properties": {
            "type": {
              "const": "object"
            }
          }
        },
        "then": {
          "required": [
            "properties"
          ]
        }
      }
    }
  }
}


Here's an example to generate json schema using descriptive language, and the return with, and without the structured output formatter.

In [23]:
json_fmt = pf_formatter.execute(user_description = "'student' is a list of dictionary, each contains exactly 2 keys: 'name' and 'age'. 'name' is a string and 'age' is an integer. Add another GPA field also", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
json_fmt

'{\n  "name": "Student_List_Format",\n  "schema": {\n    "$schema": "http://json-schema.org/draft-07/schema#",\n    "type": "object",\n    "additionalProperties": false,\n    "properties": {\n      "student": {\n        "type": "array",\n        "items": {\n          "type": "object",\n          "properties": {\n            "name": {\n              "type": "string"\n            },\n            "age": {\n              "type": "integer"\n            },\n            "GPA": {\n              "type": "number"\n            }\n          },\n          "required": ["name", "age", "GPA"],\n          "additionalProperties": false\n        }\n      }\n    },\n    "required": ["student"]\n  }\n}'

In [24]:
json.loads(json_fmt)

{'name': 'Student_List_Format',
 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'type': 'object',
  'additionalProperties': False,
  'properties': {'student': {'type': 'array',
    'items': {'type': 'object',
     'properties': {'name': {'type': 'string'},
      'age': {'type': 'integer'},
      'GPA': {'type': 'number'}},
     'required': ['name', 'age', 'GPA'],
     'additionalProperties': False}}},
  'required': ['student']}}

In [28]:
story = '''
The Fox and the Hedgehog
A fox knows many things, but a hedgehog knows one big thing.

The story
Upon crossing a river, a Fox gets his tail entangled in a bush and cannot move. While the trapped and in some dismay, a swarm of mosquitoes settle on the poor Fox and tuck into a hearty meal, unable to be swished away by the vulpine’s ensnared tail. 

‘You are in a bad way, neighbour,’ says a passing Hedgehog, seeing the Fox’s predicament. Trying to help, the Hedgehog suggests that he drive the mosquitoes away who are sucking his blood. But the wise Fox realises that by ridding him of these insects, who’ve already had their fill, would only lead to more mosquitoes arriving with a fresh appetite…and so refuses. 

The moral
Better to bear a lesser evil than to risk a greater in removing it.
'''

json_fmt = pf_formatter.execute(user_description = f"generate a JSON schema for extracting essential setups the following story, such that rebuilding it from the json can generate a story that well resemble the original story:: \n{story}", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
json.loads(json_fmt)

{'name': 'Story_Schema',
 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'type': 'object',
  'additionalProperties': False,
  'properties': {'title': {'type': 'string',
    'description': 'The title of the story.'},
   'characters': {'type': 'array',
    'description': 'The characters involved in the story.',
    'items': {'type': 'object',
     'properties': {'name': {'type': 'string',
       'description': 'The name of the character.'},
      'type': {'type': 'string',
       'description': 'The type of character (e.g., animal, human).'}},
     'required': ['name', 'type']}},
   'setting': {'type': 'string',
    'description': 'The setting where the story takes place.'},
   'plot': {'type': 'string',
    'description': 'A brief description of the plot.'},
   'moral': {'type': 'string', 'description': 'The moral of the story.'}},
  'required': ['title', 'characters', 'setting', 'plot', 'moral']}}

In [39]:
pf = PromptFunction('')
response = pf.execute(ADD_BEFORE=story, model="gpt-4o-mini",fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

{'title': 'The Fox and the Hedgehog',
 'characters': [{'name': 'Fox', 'type': 'animal'},
  {'name': 'Hedgehog', 'type': 'animal'}],
 'setting': 'A riverside with a bush nearby where the Fox gets trapped.',
 'plot': 'A Fox finds himself trapped by his tail in a bush after crossing a river. While he struggles, a swarm of mosquitoes settles on him, feeding on his blood. A passing Hedgehog offers to help by driving the mosquitoes away, but the Fox wisely refuses, understanding that this would only invite more mosquitoes to come and feast on him.',
 'moral': 'Better to bear a lesser evil than to risk a greater in removing it.'}

In [40]:
pf = PromptFunction('')
response = pf.execute(ADD_BEFORE="The student name is John and age is 20. Unfortunately, his GPA is 2.7 ", model="gpt-4o-mini",fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

{'title': 'The Turning Point',
 'characters': [{'name': 'John', 'type': 'human'},
  {'name': 'Mr. Smith', 'type': 'human'},
  {'name': 'Sarah', 'type': 'human'}],
 'setting': 'A college campus with a mix of lecture halls, a library, and a coffee shop.',
 'plot': 'John, a 20-year-old college student, struggles with his GPA of 2.7, which is affecting his confidence and future prospects. After a disappointing semester, he meets Mr. Smith, a mentor who encourages him to set realistic goals and develop a study plan. With the help of Sarah, a fellow student, John learns effective study techniques and time management skills. As the semester progresses, he begins to see improvements in his grades and rebuilds his confidence. By the end of the year, John raises his GPA and realizes that hard work and perseverance can lead to success.',
 'moral': 'With determination and the right support, anyone can overcome challenges and achieve their goals.'}

In [37]:
pf = PromptFunction('')
pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",)


"The weather in New York is rainy, with a temperature of 42°F, and it's partially cloudy."

In [41]:
response = pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",
           fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

{'title': 'Rainy Day in New York',
 'characters': [{'name': 'Alice', 'type': 'human'},
  {'name': 'Bob', 'type': 'human'},
  {'name': 'Mr. Whiskers', 'type': 'animal'}],
 'setting': 'New York City, on a rainy day, temperature 42F, partially cloudy.',
 'plot': 'Alice and Bob are best friends who decide to explore New York City despite the rain. They meet Mr. Whiskers, a clever cat who helps them find the best indoor attractions to enjoy while keeping dry.',
 'moral': 'Sometimes, you can find joy and adventure even in the most unexpected circumstances.'}

## Function calling

In [ ]:
pf_func_call_formatter = PromptFunction('''
### Task: Generate Function Calling Definitions

As an expert in creating function calling objects for auto-function invocation, your task is to generate a list of function calling definitions based on the provided user description. Follow these instructions:

1. **Use the Provided User Description**: Utilize the placeholder `user_description` to tailor the function definitions. The description should inform what functions are needed.

2. **Generate a List of Functions**: The output must be a JSON array (list) where each element is a JSON object with the following structure:
   - `"type"`: This key must have the string value `"function"`.
   - `"function"`: This is an object that must include:
     - `"name"`: A unique function name (string).
     - `"description"`: A brief description of the function's purpose (string).
     - `"parameters"`: A JSON Schema object that defines the expected input parameters. It must include:
         - `"type"`: Must be `"object"`.
         - `"properties"`: An object detailing each parameter’s type and description.
         - `"required"`: An array listing the names of required parameters.
         - `"additionalProperties"`: A boolean value (true or false) indicating whether additional properties are allowed.

3. **Output Only Valid JSON**: The final output must be valid JSON with no additional text, markdown, or explanation. It should be a JSON list of function definitions exactly as specified.

**IMPORTANT**: Your final output must be a valid JSON array of function calling definitions, where each definition includes the keys `"type"` and `"function"` (with `"name"`, `"description"`, and `"parameters"` defined as per the instructions).

### User Description

<<<user_description>>>

''')

## GPT Auto-Func Calling formatter

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "generate_json_schema",
        "description": (
            "Generate a JSON schema based on a user description. "
            "The output must be a valid JSON object that contains the keys 'name' and 'schema'. "
            "The 'schema' must follow JSON Schema Draft-07, define type 'object', "
            "allow additional properties, and incorporate the structure, types, constraints, and required fields as described."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "user_description": {
                    "type": "string",
                    "description": (
                        "A detailed description of the desired JSON schema. "
                        "Include information about the structure, types, constraints, and required fields. "
                        "For example: 'I need the JSON schema to have exactly 4 fields: weather, temperature in C, temperature in F, and location.'"
                    )
                }
            },
            "required": ["user_description"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

## Chain of Thoughts

In [58]:
pf_task_chain_planner = PromptFunction('''
You are given the following goal statement:
<<<task>>>

Task: Decompose the above goal into a comprehensive list of main actionable tasks without generating any subtasks. Provide a numbered list of tasks, where each task is clearly defined in terms of its function and prerequisites. The output should be optimized for machine parsing and contain only the main tasks, with no hierarchical breakdown.
                                       ''')

In [62]:
plan = pf_task_chain_planner.execute(task = "Calculate cosine of 24 degrees.")
print(plan)

1. Identify the method for calculating the cosine of an angle.
2. Gather necessary tools or resources for calculation (e.g., calculator, software, or mathematical tables).
3. Set the calculator or software to the correct mode (degrees).
4. Input the angle value (24 degrees) into the calculator or software.
5. Execute the calculation to determine the cosine of 24 degrees.
6. Record the result of the calculation.


In [63]:
pf_prompt_generator = PromptFunction(load_prompt("prompts/openai-prompt_generator.md"))
print(pf_prompt_generator.prompt)

Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples 

In [65]:
pf_prompt_generator.execute(user_description = "Generate a prompt that can be used write json schema.")

'Create a structured data output based on the provided input information.\n\nGiven a dataset of items, your task is to extract key attributes and organize them into a structured format. You should analyze the input details carefully to identify relevant attributes and their corresponding values.\n\n# Steps\n\n1. **Identify Key Attributes**: Review the input data to determine the most important characteristics that need to be extracted. Common attributes may include name, category, price, and description.\n2. **Extract Values**: For each identified attribute, extract the appropriate value from the input data.\n3. **Structure Data**: Organize the extracted attributes and values into a clear and concise format, ensuring that each entry is easy to read and understand.\n\n# Output Format\n\nThe output should be formatted as a JSON object, where each item is represented as a separate object within an array. Each object should contain the identified key attributes as keys and their extracted 